<a href="https://colab.research.google.com/github/William-Schwarz/Decifra-Contrato/blob/william_ds/Decifra_Contrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:

!pip -q install google-genai google-adk PyPDF2 python-docx

In [133]:
import os
from google.api_core import exceptions
from google.colab import userdata
from google import genai
from google.colab import files
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import PyPDF2

warnings.filterwarnings("ignore")

In [134]:
# Configura a API Key do Google Gemini
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    print(f"⚠️ Erro ao configurar a chave da API do Google Gemini: {e}")
    exit()

In [135]:
# Configura o cliente da SDK do Gemini
try:
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash"
except Exception as e:
    print(f"⚠️ Erro ao inicializar o cliente Gemini: {e}")
    exit()

In [136]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        final_response += "\n"
    except exceptions.ServiceUnavailable as e:
        return f"⚠️ Serviço indisponível ao comunicar com o agente '{agent.name}'. Por favor, tente novamente mais tarde. Detalhes: {e}"
    except exceptions.InvalidArgument as e:
        return f"⚠️ Argumento inválido fornecido ao agente '{agent.name}'. Verifique a entrada. Detalhes: {e}"
    except Exception as e:
        return f"⚠️ Erro ao comunicar com o agente '{agent.name}': {e}"
    return final_response

In [137]:
# Função auxiliar que faz a leitura do arquivo
def ler_arquivo(nome_arquivo):
    try:
        if nome_arquivo.endswith(".pdf"):
            with open(nome_arquivo, 'rb') as file:
                try:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page_num in range(len(reader.pages)):
                        page = reader.pages[page_num]
                        text += page.extract_text()
                    return text
                except Exception as e:
                    return f"⚠️ Erro ao ler o conteúdo do arquivo PDF '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".docx"):
            try:
                from docx import Document
                document = Document(nome_arquivo)
                text = ""
                for paragraph in document.paragraphs:
                    text += paragraph.text + "\n"
                return text
            except ImportError:
                return "⚠️ A biblioteca 'python-docx' não está instalada. Por favor, instale-a com: pip install python-docx"
            except Exception as e:
                return f"⚠️ Erro ao ler o conteúdo do arquivo DOCX '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".txt"):
            with open(nome_arquivo, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return "⚠️ Formato de arquivo não suportado. Por favor, envie um arquivo .pdf, .docx ou .txt."
    except FileNotFoundError:
        return f"⚠️ Erro: Arquivo '{nome_arquivo}' não encontrado."
    except Exception as e:
        return f"⚠️ Erro ao processar o arquivo '{nome_arquivo}': {e}"

In [138]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [139]:
# Agente 1: Receptor e Extrator de Contrato
def agente_receptor():
    receptor = Agent(
        name="agente_receptor",
        model=MODEL_ID,
        instruction="""
        Você é o agente inicial do sistema Decifra-Contrato. Sua tarefa é receber o contrato fornecido pelo usuário.
        Este contrato pode ser um texto diretamente inserido ou o conteúdo de um arquivo.

        Uma vez que o contrato é fornecido, sua principal função é extrair o texto completo e limpo do contrato.
        Remova qualquer formatação desnecessária que possa dificultar a análise posterior (como quebras de linha excessivas, numerações automáticas, etc.).

        Se o usuário fornecer informações adicionais sobre o tipo de contrato (por exemplo, "contrato de aluguel", "termo de serviço"), anote essa informação para auxiliar os próximos agentes.

        Ao final, entregue o texto puro do contrato para o próximo agente.
        """,
        description="Agente que recebe e extrai o texto do contrato."
    )
    return receptor

In [140]:
# Agente 2: Identificador de Jargões
def agente_identificador():
    identificador = Agent(
        name="agente_identificador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em identificar jargões jurídicos e termos técnicos dentro de um texto de contrato.
        Sua tarefa é analisar o texto do contrato fornecido e listar todos os termos que provavelmente seriam desconhecidos por uma pessoa leiga.

        Considere termos como: latim jurídico (e.g., 'ad hoc'), termos técnicos específicos da área do contrato (e.g., em um contrato de tecnologia, 'API', 'SaaS'), e cláusulas com redação complexa.

        Apresente a lista de jargões identificados de forma clara e concisa para o próximo agente.
        """,
        description="Agente que identifica jargões jurídicos e técnicos."
    )
    return identificador

In [141]:
# Agente 3: Explicador de Termos
def agente_explicador(google_search):
    explicador = Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em explicar termos jurídicos e técnicos de forma clara e acessível para pessoas leigas.
        Você receberá uma lista de jargões identificados em um contrato.

        Para cada termo da lista, sua tarefa é fornecer uma explicação concisa e em linguagem simples.
        Se necessário, utilize a ferramenta de busca do Google (google_search) para encontrar definições e explicações adequadas.

        Tente fornecer exemplos práticos ou analogias para facilitar a compreensão.

        Apresente cada termo seguido de sua explicação de forma clara e organizada.
        """,
        description="Agente que explica jargões jurídicos e técnicos para leigos.",
        tools=[google_search]
    )
    return explicador

In [142]:
# Agente 4: Resumidor e Destacador de Pontos Chave
def agente_resumidor():
    resumidor = Agent(
        name="agente_resumidor",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em resumir contratos e destacar os pontos mais importantes para uma pessoa leiga.
        Com base no texto completo do contrato, sua tarefa é gerar um resumo conciso, com linguagem simples e direta.

        Além do resumo, identifique e liste as cláusulas ou pontos que são cruciais para o entendimento do usuário, como:
        - Objeto do contrato
        - Obrigações das partes
        - Prazos e condições
        - Formas de pagamento (se aplicável)
        - Condições de rescisão
        - Penalidades (se houver)

        Apresente o resumo e os pontos chave de forma clara e organizada, utilizando marcadores ou listas para facilitar a leitura.
        """,
        description="Agente que resume contratos e destaca pontos chave."
    )
    return resumidor

In [144]:
# Execução

print("📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐")

contrato_texto = ""
while True:
    opcao = input("❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? ").lower()
    if opcao in ['texto', 'arquivo']:
        break
    else:
        print("\n⚠️ Opção inválida. Por favor, digite 'texto' ou 'arquivo'.")

if opcao == 'texto':
    contrato_texto = input("Cole o texto do contrato aqui: ")
elif opcao == 'arquivo':
    try:
        uploaded = files.upload()
        if uploaded:
            nome_arquivo = list(uploaded.keys())[0]
            contrato_texto = ler_arquivo(nome_arquivo)
            if isinstance(contrato_texto, str) and contrato_texto.startswith("⚠️ Erro"):
                print(f"\n{contrato_texto}")
                contrato_texto = "" # Limpa para não prosseguir com erro
            elif isinstance(contrato_texto, str) and contrato_texto.startswith("⚠️ Formato de arquivo não suportado"):
                print(f"\n{contrato_texto}")
                contrato_texto = "" # Limpa para não prosseguir com erro
            else:
                print(f"\nArquivo '{nome_arquivo}' lido com sucesso.")
        else:
            print("\n⚠️ Nenhum arquivo foi carregado.")
    except Exception as e:
        print(f"\n⚠️ Ocorreu um erro ao carregar o arquivo: {e}")
        contrato_texto = ""

    # Inicializa os agentes
    try:
        receptor = agente_receptor()
        identificador = agente_identificador()
        explicador = agente_explicador(google_search)
        resumidor = agente_resumidor()
    except Exception as e:
        print(f"⚠️ Erro ao inicializar os agentes: {e}")
        contrato_texto = "" # Cancela o processamento

    if contrato_texto:
        display(Markdown("## ✨ Processando..."))
        texto_contrato_extraido = call_agent(receptor, contrato_texto if opcao == 't' else f"Arquivo: {contrato_texto}")
        if texto_contrato_extraido.startswith("⚠️"):
            display(Markdown(f"### ⚠️ Extração: <span style='color:orange'>{texto_contrato_extraido}</span>"))
        else:
            display(Markdown("### 📝 Contrato Extraído"))
            display(to_markdown(texto_contrato_extraido))

            jargoes_identificados = call_agent(identificador, texto_contrato_extraido)
            display(Markdown("### 🔎 Jargões"))
            display(to_markdown(jargoes_identificados))

            explicacoes_termos = call_agent(explicador, jargoes_identificados)
            display(Markdown("### 💡 Explicações"))
            display(to_markdown(explicacoes_termos))

            resumo_contrato = call_agent(resumidor, texto_contrato_extraido)
            display(Markdown("### 🔑 Resumo e Pontos Chave"))
            display(to_markdown(resumo_contrato))

            display(Markdown("## ✅ Análise Concluída!"))
else:
    print("Processamento cancelado devido à falta de contrato ou à mà qualidade do arquivo.")

📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐
❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? arquivo


Saving Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS.pdf to Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS (6).pdf

Arquivo 'Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS (6).pdf' lido com sucesso.


## ✨ Processando...

### 📝 Contrato Extraído

> Aqui está o texto extraído e limpo do contrato de prestação de serviços:
> 
> CONTRATO DE PRESTAÇÃO DE SERVIÇOS
> 
> CONTRATANTE: ____________________________, inscrito CPF sob nº ________________ e RG: _________ SSP ___, a Rua __________, nº _______, bairro _____________, Florianópolis/SC, diretor do curta-metragem “_____________________”.
> CONTRATADO:_______________________________, inscrito CPF sob nº _________________, sito à Rua ______________________________________________________________, Florianópolis/SC, ____________________do curta-metragem “___________________”.
> 
> CONTRATANTE e CONTRATADO, acima nomeados e qualificados, tem entre si justo e acordado o seguinte:
> 
> O CONTRATANTE, como principal responsável, está iniciando a realização de obra audiovisual cinematográfica de CURTA-METRAGEM EM VÍDEO, previamente intitulado _________________________, sob direção de ______________________, doravante denominada simplesmente OBRA.
> 
> CLÁUSULA PRIMEIRA - DO OBJETO E DAS OBRIGAÇÕES
> Tendo em vista o roteiro e a concepção artística do mencionado filme, o CONTRATANTE contrata o CONTRATADO para prestar-lhe os serviços de técnico cinematográfico na função de _____________________________________.
> 
> 1.1 A fim de garantir a boa execução do trabalho, o CONTRATADO compromete-se, por este ato, a observar e cumprir o cronograma de atividades da referida obra.
> 
> 1.2 O CONTRATADO assume as responsabilidades inerentes a sua função no filme e se dispõe a realizá-la de acordo com o que for requerido pela direção e produção da obra.
> 
> 1.3 O CONTRATADO cede, por prazo indeterminado e de forma irretratável e irrenunciável para o CONTRATANTE, todos os direitos patrimoniais e conexos relativos ao seu trabalho na citada obra, que possam ser auferidos por qualquer utilização do filme, no Brasil e no exterior, para fins comerciais ou não, incluindo a reprodução, gravação, promoção e divulgação da obra, nos meios audiovisuais competentes existentes e a serem criados.
> 
> CLÁUSULA SEGUNDA – DA DURAÇÃO DO CONTRATO
> Os serviços aqui contratados deverão ser realizados no período de ______________ , em datas previamente acertadas entre as partes.
> 
> CLÁUSULA TERCEIRA – DA REMUNERAÇÃO
> 
> 3.1 O CONTRATADO não receberá pelo trabalho qualquer ônus ou remuneração.
> 
> CLÁUSULA QUARTA – DOS CRÉDITOS
> 4.1 De acordo com o estabelecido na legislação vigente, o CONTRATADO terá o direito de que figure seu nome nos créditos da OBRA na função referida neste contrato.
> 
> CLÁUSULA QUINTA – DA RESOLUÇÃO
> 5.1 Em caso de extinção do presente contrato, em qualquer uma das formas, os trabalhos realizados pelo CONTRATADO e os direitos autorais cedidos na vigência do mesmo, serão utilizados pela CONTRATANTE, ressalvados os direitos de cunho moral relativos ao presente.
> 
> CLÁUSULA SEXTA – CONVENÇÃO DAS PARTES
> 6.1 Este contrato terá vigência a partir de sua assinatura até a finalização da OBRA.
> 6.2 As partes concordam que este instrumento contem a totalidade dos entendimentos entre as partes, perdendo sua validade todos e quaisquer outros entendimentos previamente acordados entre as partes, sejam estes orais, escritos ou de que natureza forem.
> 
> E por estarem assim, justos e contratados, assinam o presente em 2 (duas) vias de igual teor e forma, para o mesmo fim.
> 
> Florianópolis, ____________________________________
> 
> ____________________________________ ________________________________
> 
> CONTRATANTE CONTRATADO
> 


...

### 💡 Explicações

> Ok, posso ajudar com isso. Aqui estão as explicações de cada termo, em linguagem simples:
> 
> *   **Obra audiovisual cinematográfica:** É basicamente um filme, como aqueles que você assiste no cinema ou em plataformas de streaming. É um termo mais técnico usado em contratos.
> 
> *   **Direitos patrimoniais e conexos:** Imagine que você criou algo, como uma música ou um filme. Os direitos patrimoniais são como o direito de ganhar dinheiro com isso, de vender, alugar ou exibir para outras pessoas. Já os direitos conexos são os direitos de quem participa da criação, como os atores, músicos e produtores.
> 
> *   **Prazo indeterminado:** É como se não houvesse data para acabar. No caso do contrato, significa que a pessoa que está cedendo os direitos não vai tê-los de volta em uma data específica.
> 
> *   **De forma irretratável e irrenunciável:** Isso quer dizer que, uma vez que a pessoa cedeu os direitos, ela não pode mudar de ideia e pegar os direitos de volta, nem abrir mão do que foi acordado.
> 
> *   **Meios audiovisuais competentes existentes e a serem criados:** É uma forma de garantir que o filme possa ser exibido em qualquer formato, seja no cinema, na TV, em DVD, em plataformas de streaming e até em tecnologias que ainda nem foram inventadas.
> 
> *   **Legislação vigente:** São as leis que valem hoje e que se aplicam ao contrato. É importante porque garante que o contrato está de acordo com as regras atuais.
> 
> *   **Extinção do presente contrato, em qualquer uma das formas:** É o fim do contrato, que pode acontecer por vários motivos, como término do projeto, quebra de contrato ou acordo entre as partes.
> 
> *   **Direitos de cunho moral:** Imagine que você criou uma obra. Os direitos morais são aqueles que protegem a sua reputação como criador e garantem que ninguém vai mexer na sua obra sem a sua permissão.
> 
> *   **Vigência:** É o tempo que o contrato vale, desde o dia em que ele começa até o dia em que ele termina.
> 
> *   **Em 2 (duas) vias de igual teor e forma, para o mesmo fim:** Significa que cada pessoa que assina o contrato recebe uma cópia idêntica, para que todos saibam exatamente o que foi combinado.
> 


### 🔑 Resumo e Pontos Chave

> ## Resumo do Contrato de Prestação de Serviços (Curta-Metragem)
> 
> Este contrato define os termos do acordo entre o diretor do curta-metragem (CONTRATANTE) e o técnico cinematográfico (CONTRATADO) para a realização do filme "[Nome do Curta-Metragem]". O CONTRATADO prestará serviços como [Função do técnico] durante o período de [Data de início] a [Data de término]. O contrato estabelece que o CONTRATADO não receberá remuneração pelo trabalho, mas terá seu nome creditado no filme. Além disso, o CONTRATADO cede ao CONTRATANTE todos os direitos sobre o seu trabalho no filme para fins de reprodução e divulgação.
> 
> **Pontos Chave do Contrato:**
> 
> *   **Objeto do Contrato:** Prestação de serviços como técnico cinematográfico na função de [Função do técnico] para o curta-metragem "[Nome do Curta-Metragem]".
> *   **Obrigações do CONTRATADO:**
>     *   Cumprir o cronograma de atividades do filme.
>     *   Realizar as tarefas de acordo com as instruções da direção e produção.
>     *   Ceder os direitos patrimoniais e conexos sobre seu trabalho no filme ao CONTRATANTE (direitos de reprodução, divulgação, etc.).
> *   **Obrigações do CONTRATANTE:**
>     *   Garantir que o nome do CONTRATADO seja incluído nos créditos do filme.
>     *   Utilizar os trabalhos realizados pelo CONTRATADO e os direitos autorais cedidos, mesmo em caso de rescisão do contrato.
> *   **Prazos e Condições:** Os serviços serão prestados no período de [Data de início] a [Data de término], em datas previamente combinadas. O contrato tem validade a partir da assinatura até a finalização do filme.
> *   **Forma de Pagamento:** O CONTRATADO não receberá qualquer remuneração pelo trabalho.
> *   **Condições de Rescisão:** O contrato pode ser rescindido, mas o CONTRATANTE poderá continuar utilizando o trabalho do CONTRATADO e os direitos cedidos.
> *   **Direitos:** O CONTRATADO tem direito aos créditos na obra.


## ✅ Concluído!